Python Imports

In [1]:
import os

import pandas as pd
import numpy as np

In [3]:
import pymysql
import pymysql.cursors

Establish database connection

In [10]:
# Connect to the database
connection = pymysql.connect(host='cis550-2.cmxt8otwhjqc.us-east-2.rds.amazonaws.com',
                             db='cis550',
                             user='cis550',
                             password='cis550eklh',
#                              charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [ ]:
# run this box after you are done with the operations
connection.close()

In [52]:
# test query
with connection.cursor() as cursor:
    # Read a single record
    sql = '''
    show tables
    ;
    '''
    cursor.execute(sql)
    result = cursor.fetchall()
    print(result)

()


<hr>

Create tables in database

In [62]:
# don't run again

# table_creation_queries = [
#     '''CREATE TABLE IF NOT EXISTS artist (
#         artist_id         VARCHAR(25) NOT NULL,
#         artist_name       VARCHAR(50) NOT NULL,
#         artist_sp_name    VARCHAR(50) NOT NULL,
#         PRIMARY KEY (artist_id)
#     );''',
#     '''CREATE TABLE IF NOT EXISTS artist_genre (
#         artist_id         VARCHAR(25) NOT NULL,
#         genre             VARCHAR(50) NOT NULL,
#         PRIMARY KEY(artist_id, genre),
#         FOREIGN KEY(artist_id) references artist(artist_id)
#     );''',
#     '''CREATE TABLE IF NOT EXISTS city (
#         city_id           INT         NOT NULL AUTO_INCREMENT,
#         city_name         VARCHAR(50) NOT NULL,
#         city_state        VARCHAR(50) NOT NULL,
#         lat               FLOAT       NOT NULL,
#         lon               FLOAT       NOT NULL,
#         PRIMARY KEY (city_id)
#     );''',
#     '''CREATE TABLE IF NOT EXISTS artist_city (
#         artist_id         VARCHAR(25) NOT NULL,
#         city_id           INT         NOT NULL,
#         arr_index         INT         NOT NULL,
#         PRIMARY KEY(artist_id, city_id, arr_index),
#         FOREIGN KEY(artist_id) references artist(artist_id),
#         FOREIGN KEY(city_id) references city(city_id)
#     );''',
#     '''CREATE TABLE IF NOT EXISTS artist_rank (
#         artist_id         VARCHAR(25) NOT NULL,
#         year              INT         NOT NULL,
#         a_rank            INT         NOT NULL,
#         PRIMARY KEY (artist_id, year, a_rank),
#         FOREIGN KEY(artist_id) references artist(artist_id)
#     );''',
# ]

# with connection.cursor() as cursor:
#     for query in table_creation_queries:
#         cursor.execute(query)
#         result = cursor.fetchall()
#         print(result)

# connection.commit()

()
()
()
()
()


In [63]:
table_verification_queries = table_creation_queries = [
    '''DESC artist;''',
    '''DESC artist_genre;''',
    '''DESC city;''',
    '''DESC artist_city;''',
    '''DESC artist_rank;'''
]

test = None
with connection.cursor() as cursor:
    
    fields = ['Field', 'Type', 'Null?', 'Key', 'Default', 'Extras']
    print('\t\t'.join(fields))
    print('\t\t'.join(['=' * len(field) for field in fields]))
    print()
    
    tables = ['artist', 'artist_genre', 'city', 'artist_city', 'artist_rank']
    
    for i in range(0, len(table_creation_queries)):
        cursor.execute(table_creation_queries[i])
        result = cursor.fetchall()
        
        print(tables[i] + '\n' + '-' * len(tables[i]))
        for r in result:
            print(*['{:15}'.format(str(x)) for x in r.values()])
        print()

Field		Type		Null?		Key		Default		Extras
=====		====		=====		===		=======		======

artist
------
artist_id       varchar(25)     NO              PRI             None                           
artist_name     varchar(50)     NO                              None                           
artist_sp_name  varchar(50)     NO                              None                           

artist_genre
------------
artist_id       varchar(25)     NO              PRI             None                           
genre           varchar(50)     NO              PRI             None                           

city
----
city_id         int(11)         NO              PRI             None            auto_increment 
city_name       varchar(50)     NO                              None                           
city_state      varchar(50)     NO                              None                           
lat             float           NO                              None                           
l

In [64]:
# query = ''''''.format(', '.join(tables))

# with connection.cursor() as cursor:
#     cursor.execute(query)
#     result = cursor.fetchall()
#     print(result)

<hr>

Load data

In [75]:
billboard_file = 'clean_billboard.csv'
billboard_df = pd.read_csv(billboard_file)
billboard_df

,a_rank,year,artist_id
0,1,2018,3TVXtAsR1Inumwj472S9r4
1,9,2018,3TVXtAsR1Inumwj472S9r4
2,69,2018,3TVXtAsR1Inumwj472S9r4
3,1,2018,3TVXtAsR1Inumwj472S9r4
4,9,2018,3TVXtAsR1Inumwj472S9r4
5,69,2018,3TVXtAsR1Inumwj472S9r4
6,1,2018,3TVXtAsR1Inumwj472S9r4
7,7,2018,3TVXtAsR1Inumwj472S9r4
8,59,2018,3TVXtAsR1Inumwj472S9r4
9,1,2018,3TVXtAsR1Inumwj472S9r4


In [76]:
artist_genre_file = 'artists_genres_cleaned.csv'
artist_genre_df = pd.read_csv(artist_genre_file)
artist_genre_df

,found_name,genres,read_name,spotify_id
0,"Richard ""Groove"" Holmes","['jazz funk', 'jazz organ', 'soul jazz']","""Groove"" Holmes",7izkpvBGc31hUccjdSNz8k
1,"""Weird Al"" Yankovic","['comedy rock', 'comic']","""Weird Al"" Yankovic",1bDWGdIC2hardyt55nlQgG
2,$uicideBoy$,['underground hip hop'],$uicideBoy$,1VPmR4DJC1PlOtd0IADAO0
3,'In The Heights' Original Broadway Company,"['broadway', 'hollywood']",'In The Heights' Original Broadway Company,0kvp9mzfvoXvGtjSWTgrEb
4,*NSYNC,"['boy band', 'dance pop', 'europop', 'pop', 'p...",'N Sync,6Ff53KvcvAj5U7Z1vojB5o
5,'Til Tuesday,"['boston rock', 'dance rock', 'new romantic', ...",'Til Tuesday,1L0y9srZMyh9XUnYGv37IP
6,+44,"['emo', 'pop punk']",+44,7gkRNHOOt7QfhhXf0rEnmj
7,10 Years,"['alternative metal', 'christian alternative r...",10 Years,0REMf7H0VP6DwfZ9MbuWph
8,"10,000 Maniacs","['alternative rock', 'folk', 'folk-pop', 'lili...","10,000 Maniacs",0MBIKH9DjtBkv8O3nS6szj
9,10cc,"['album rock', 'art rock', 'classic rock', 'da...",10cc,6i6WlGzQtXtz7GcC5H5st5


Have to flatten the genre column.

TODO:
- artist_genre
    - select and rename
    - flatten the genre column
- other tables

<hr>

Upload to Amazon RDS database

In [ ]:
'INSERT INTO {table} ({fields}) VALUES'
    {values},
    {values},
    {values},
    {values},
    ...